# TE Academy Reputation Weighted Voting

## **Nakamoto-1.0**

### Exploring the Nakamoto Coefficient for a Specific Set of Parameters


In this notebook we examine three candidate mechanisms to explore their Nakamoto coefficient. 

## Process

For each mechanism, we will assign voter information based on a recent capture of voter NFT information, and an initial choice of NFT weights selected by TE Academy. 

We then do the following:
- Sort voters according to NFT weights, based on the weights. 
- Create an election where all candidates vote for `candidate_A`.
- Going down the voters list based on weights, we do the following steps:
        * **Step 1:** Change the voters vote to `candidate_B`. 
        * **Step 2:** See if this changes the result of the election. 
        * **Step 3:** If it does, we stop. If not, we select the next voter and return to Step 1. 

The minimum number of voters needed to flip the election is called the **Nakamoto Coefficient**, and represents the minimum ``coalition'' size that is needed to influence the election. 

## Importing Code

We begin by importing necessary Python code. 

In [1]:
import pandas as pd
import sys
sys.path.append("..")

from copy import deepcopy
from math import ceil
from IPython.display import display, Markdown

from mechanisms.group_hug_mechanism import GroupHug
from mechanisms.rank_n_slide_mechanism import RankAndSlide
from mechanisms.single_choice_qcv_mechanism import SingleChoiceQuadraticCredibility

## Importing Voter Data

In the file `nft_data_may_28_2024_cleaned.csv` in the `data` folder, there is information pulled from the OtterSpace subgraph related to TE Academy NFT data. 

WWe can use this data to better understand the mechanisms. 

In [2]:
voter_data =  pd.read_csv("../data/nft_data_may_28_2024_cleaned.csv")
voter_data.head(5)

,Unnamed: 0,ID,NFTREP_V1,SPEAKER_ETHCC_PARIS23,STUDY_GROUP_HOST_C2_22_23,ETHCC_23,FUND_AUTHOR,STUDY_GROUP_HOST_360_22,STUDY_GROUP_HOST_FUND_22_23,SPEAKER_BARCAMP_PARIS_23,BARCAMP_PARIS_23,TEAM_BARCAMP_PARIS_23,FUND_MOD_1,FUND_MOD_2,FUND_MOD_3,FUND_MOD_4,FUND_MOD_5
0,0,0xbb8743ea733155fe5e81ed285aea72cc19b2ca87,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,1,0x049debbad61a20e21e872b06ef4f25be1253c802,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1
2,2,0xa481db1ac55683dfb2847c02e417b7e411bcbbea,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1
3,3,0x024bd3d1c3b6ba2277a744d7a99fbade9404a370,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,4,0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1


We do a bit of preprocessing to convert the voter information to a dictionary format that we need. 

In the final nested dictionary:
* the keys are strings corresponding to wallet IDs
* the values are dictionaries, which give the NFT information for each wallet

In [3]:
# We may need to drop a specific column. 
voter_data.drop(columns = ['Unnamed: 0'], 
                inplace = True)
voter_data.set_index('ID', inplace = True)

In [4]:
voter_dict = voter_data.to_dict(orient='index')

Any time we need to refer to voter NFT information, we can use this `voter_dict` variable. Below is an example of *one* voter. 

## Reading NFT Weights

We use the voting weights suggested as potential initial choices from TE Academy, which are stored in the `default_voting_weights.csv` file in the `data` directory. 

In [5]:
nft_weights = pd.read_csv("../data/default_voting_weights.csv")

We see a small sample here. 

In [6]:
nft_weights.head(5)

,Name,CodeName,Category,Count,Weight
0,TE FUNDAMENTALS COURSE AUTHOR LAUNCH 2022,FUND_AUTHOR,POE,4.0,20.0
1,Token Engineering @EthCC Paris 2023 - Speaker,SPEAKER_ETHCC_PARIS23,POE,7.0,16.0
2,TE FUNDAMENTALS 1,FUND_MOD_1,POK,332.0,7.0
3,TE FUNDAMENTALS 2,FUND_MOD_2,POK,249.0,7.0
4,TE FUNDAMENTALS 3,FUND_MOD_3,POK,168.0,7.0


We need to do a small bit of processing to get this information into a dictionary.

In [7]:
# Convert csv file to dictionary
nft_weights_dict = {
    nft_weights.at[k,"CodeName"]: nft_weights.at[k,"Weight"]
    for k in range(len(nft_weights))
}

In [8]:
nft_weights_dict

{'FUND_AUTHOR': 20.0,
 'SPEAKER_ETHCC_PARIS23': 16.0,
 'FUND_MOD_1': 7.0,
 'FUND_MOD_2': 7.0,
 'FUND_MOD_3': 7.0,
 'FUND_MOD_4': 7.0,
 'FUND_MOD_5': 7.0,
 'NFTREP_V1': 3.0,
 'ETHCC_23': 1.0,
 'SPEAKER_BARCAMP_PARIS_23': 12.0,
 'BARCAMP_PARIS_23': 5.0,
 'TEAM_BARCAMP_PARIS_23': 1.0,
 'STUDY_GROUP_HOST_FUND_22_23': 10.0,
 'STUDY_GROUP_HOST_C2_22_23': 10.0,
 'STUDY_GROUP_HOST_360_22': 10.0,
 'STUDY_SEASON_REGISTRATION': 1.0,
 'LIVE_TRACK_1': 10.0,
 'LIVE_TRACK_3': 10.0,
 'LIVE_TRACK_4': 10.0,
 'LIVE_TRACK_5': 10.0,
 'LIVE_TRACK_6': 10.0,
 'LIVE_TRACK_7': 10.0,
 'LIVE_TRACK_8': 10.0,
 'FELLOWSHIP_COMM': 16.0,
 'STUDY_SEASON_SPEAKER': 16.0,
 'FUND_WE_MADE_IT': 10.0,
 'FUND_MOD_3_AND_4': 5.0,
 'FUND_ALL': 10.0}

## Sorting The Voters By Weight

For our purposes, it will be important to have the voters sorted by how much "raw" voting power they have based on their NFTs. Strictly speaking, this may not correspond to how much influence they have, if their votes are processed or combined in some way (like in Quadratic Credibility Voting or GroupHug).

However, we are lucky that all of the methods considered have the property of [monotonicity](https://en.wikipedia.org/wiki/Monotonicity_criterion). So, sorting for Nakamoto Coefficient in this way works well. 

In [9]:
for voter in voter_dict.keys():
    voter_dict[voter]['total_weight'] = 0
    for nft in voter_dict.get(voter).keys():
        voter_dict[voter]['total_weight'] += voter_dict.get(voter).get(nft) * nft_weights_dict.get(nft, 0)

sorted_voter_dict = {k: voter_dict[k] 
                     for k 
                     in sorted(voter_dict, 
                               key=lambda x: voter_dict[x]['total_weight'],
                               reverse=True)}

In [10]:
sorted_voter_dict

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 1,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 1,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 1,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 1,
  'FUND_MOD_1': 1,
  'FUND_MOD_2': 1,
  'FUND_MOD_3': 1,
  'FUND_MOD_4': 1,
  'FUND_MOD_5': 0,
  'total_weight': 77.0},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 1,
  'SPEAKER_BARCAMP_PARIS_23': 1,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 1,
  'FUND_MOD_1': 1,
  'FUND_MOD_2': 1,
  'FUND_MOD_3': 1,
  'FUND_MOD_4': 1,
  'FUND_MOD_5': 1,
  'total_weight': 58.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'NFTREP_V1': 1,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_2

Since we are doing copy-paste/template work, many of the variable names below are hard-coded. To simplify things later on, we will re-use the `voter_dict` variable name to store a copy of `sorted_voter_dict`.

In [11]:
voter_dict = deepcopy(sorted_voter_dict)

In [12]:
voter_dict

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 1,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 1,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 1,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 1,
  'FUND_MOD_1': 1,
  'FUND_MOD_2': 1,
  'FUND_MOD_3': 1,
  'FUND_MOD_4': 1,
  'FUND_MOD_5': 0,
  'total_weight': 77.0},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 1,
  'SPEAKER_BARCAMP_PARIS_23': 1,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 1,
  'FUND_MOD_1': 1,
  'FUND_MOD_2': 1,
  'FUND_MOD_3': 1,
  'FUND_MOD_4': 1,
  'FUND_MOD_5': 1,
  'total_weight': 58.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'NFTREP_V1': 1,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_2

## Checking Nakamoto Coefficient for Quadratic Credibility Voting

We define an instance of the Single Choice Quadratic Credibility mechanism.

In [13]:
scqc_mechanism = SingleChoiceQuadraticCredibility()

We give the voters their points according to those NFT weights, distributed as the SingleChoiceQuadraticCredibility mechanism will. 

In [14]:
scqc_voters_points = scqc_mechanism.allocate_points_from_credentials(
    voter_credentials = voter_dict,
    credential_weights = nft_weights_dict,
    total_amount_to_allocate = 10_000
)

In [15]:
scqc_voters_points

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'points': 134.3804537521815},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'points': 101.22164048865619},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'points': 97.73123909249564},
 '0xcc449df434d886576202eef7c9ef2aa0948c310b': {'points': 83.76963350785341},
 '0x0e950e75b652878e6a3d3ecb6e5797452a918a05': {'points': 82.02443280977313},
 '0x148d424f74c323d3950a8562760aa76e4c58615b': {'points': 80.27923211169285},
 '0xf9278a270d2100ea5e76e8d85ac712b2cb7ec685': {'points': 78.53403141361257},
 '0x47f793e8af738f4b61d7e2d207c6d29c713d1a84': {'points': 76.78883071553228},
 '0x6d465d2081b799770d0ce7e755d8db1665903ffb': {'points': 68.06282722513089},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'points': 66.3176265270506},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'points': 66.3176265270506},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'points': 66.3176265270506},
 '0xc8eba9130f40a0ea9be530407e5b4f7199118864': {'points': 66.31762

We create a reference dictionary of vote information, where all candidates vote for candidate_A.

In [16]:
all_votes_for_A = {voter: {"candidate_A": scqc_voters_points.get(voter).get("points")}
                           for voter in scqc_voters_points.keys() }

This is what this initial dictionary looks like. 

In [17]:
all_votes_for_A

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'candidate_A': 134.3804537521815},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'candidate_A': 101.22164048865619},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'candidate_A': 97.73123909249564},
 '0xcc449df434d886576202eef7c9ef2aa0948c310b': {'candidate_A': 83.76963350785341},
 '0x0e950e75b652878e6a3d3ecb6e5797452a918a05': {'candidate_A': 82.02443280977313},
 '0x148d424f74c323d3950a8562760aa76e4c58615b': {'candidate_A': 80.27923211169285},
 '0xf9278a270d2100ea5e76e8d85ac712b2cb7ec685': {'candidate_A': 78.53403141361257},
 '0x47f793e8af738f4b61d7e2d207c6d29c713d1a84': {'candidate_A': 76.78883071553228},
 '0x6d465d2081b799770d0ce7e755d8db1665903ffb': {'candidate_A': 68.06282722513089},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'candidate_A': 66.3176265270506},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'candidate_A': 66.3176265270506},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'candidate_A': 66.3176265270506},
 '0xc8

In [18]:
#NOTE: This function could be refactored to take in the NFT weights at the outset
# for easier experimentation. 
 
def find_nakamoto_coefficient_scqc(quad_mechanism,
                           voter_points_dict):
    # Create reference dictionary with all votes for A
    all_votes_for_A = {voter: {"candidate_A": voter_points_dict.get(voter).get("points")}
                           for voter in voter_points_dict.keys() }
    # Create an identical dictionary 
    B_gaining_votes = {voter: {"candidate_A": voter_points_dict.get(voter).get("points")}
                           for voter in voter_points_dict.keys() }
    
    num_voters_for_B = 0

    for voter in list(voter_points_dict.keys()):
        # Take the default "everyone votes for A" election
        # and change one vote to  at a time. 
        B_gaining_votes[voter] = {"candidate_B": voter_points_dict.get(voter).get("points") }
        # Increase the count of how many voters voted for B
        num_voters_for_B = num_voters_for_B + 1
                    
        winner, results = quad_mechanism.calculate(voters = voter_points_dict,
                                              voter_choices = B_gaining_votes)
        
        if winner == "candidate_B":
            half_total_number_voters = ceil(0.5 * len(voter_points_dict))
            nakamoto_coefficient = min(half_total_number_voters,
                                       num_voters_for_B)
            return nakamoto_coefficient, B_gaining_votes
        
    return nakamoto_coefficient, B_gaining_votes

In [19]:
qc_nakamoto_coefficient, final_B_votes = find_nakamoto_coefficient_scqc(quad_mechanism = scqc_mechanism,
                       voter_points_dict = scqc_voters_points)

In [20]:
print(f"The nakamoto coefficient of Quadratic Credibility is {qc_nakamoto_coefficient}.")

The nakamoto coefficient of Quadratic Credibility is 120.


In [34]:
120/len(scqc_voters_points)

0.345821325648415

In [21]:
final_B_votes

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'candidate_B': 134.3804537521815},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'candidate_B': 101.22164048865619},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'candidate_B': 97.73123909249564},
 '0xcc449df434d886576202eef7c9ef2aa0948c310b': {'candidate_B': 83.76963350785341},
 '0x0e950e75b652878e6a3d3ecb6e5797452a918a05': {'candidate_B': 82.02443280977313},
 '0x148d424f74c323d3950a8562760aa76e4c58615b': {'candidate_B': 80.27923211169285},
 '0xf9278a270d2100ea5e76e8d85ac712b2cb7ec685': {'candidate_B': 78.53403141361257},
 '0x47f793e8af738f4b61d7e2d207c6d29c713d1a84': {'candidate_B': 76.78883071553228},
 '0x6d465d2081b799770d0ce7e755d8db1665903ffb': {'candidate_B': 68.06282722513089},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'candidate_B': 66.3176265270506},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'candidate_B': 66.3176265270506},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'candidate_B': 66.3176265270506},
 '0xc8

Since the list is empty, this mechanism and this parameter setting have no dictators. 

## Checking Rank-and-Slide for Dictators

In [22]:
rs_mechanism = RankAndSlide()

In [23]:
rank_and_slide_voters_points = rs_mechanism.allocate_points_from_credentials(
    voter_credentials=voter_dict,
    credential_weights=nft_weights_dict
)

In [24]:
rank_and_slide_voters_points

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'points': 77.0},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'points': 58.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'points': 56.0},
 '0xcc449df434d886576202eef7c9ef2aa0948c310b': {'points': 48.0},
 '0x0e950e75b652878e6a3d3ecb6e5797452a918a05': {'points': 47.0},
 '0x148d424f74c323d3950a8562760aa76e4c58615b': {'points': 46.0},
 '0xf9278a270d2100ea5e76e8d85ac712b2cb7ec685': {'points': 45.0},
 '0x47f793e8af738f4b61d7e2d207c6d29c713d1a84': {'points': 44.0},
 '0x6d465d2081b799770d0ce7e755d8db1665903ffb': {'points': 39.0},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'points': 38.0},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'points': 38.0},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'points': 38.0},
 '0xc8eba9130f40a0ea9be530407e5b4f7199118864': {'points': 38.0},
 '0x6e3556f2c8f29559ed67642a1813b177f1871613': {'points': 38.0},
 '0x278a4bd5a8d09bcf11b1052ec3129a7efa75d213': {'points': 38.0},
 '0x28ddb1f987f1d433eb82e

In [25]:
#NOTE: This function could be refactored to take in the NFT weights at the outset
# for easier experimentation. 
 
def find_nakamoto_rank_and_slide(rank_and_slide_mechanism,
                           voter_points_dict):
    
        
    # Track how many people have voted for B
    num_voters_for_B = 0
    
    # Create reference dictionary with all votes for A
    all_votes_for_A = {voter: {"candidate_A": voter_points_dict.get(voter).get("points")}
                           for voter in voter_points_dict.keys() }
    # Create an identical dictionary 
    B_gaining_votes = {voter: {"candidate_A": voter_points_dict.get(voter).get("points")}
                           for voter in voter_points_dict.keys() }

    for voter in list(voter_points_dict.keys()):
        # Take the default "everyone votes for A" election
        # and change one vote to B
        B_gaining_votes[voter] = {"candidate_B": voter_points_dict.get(voter).get("points") }

        # Increase the count of how many voters voted for B
        num_voters_for_B = num_voters_for_B + 1
                    
        winner, results = rank_and_slide_mechanism.calculate(voters = voter_points_dict,
                                              voter_choices = B_gaining_votes)
        if winner == "candidate_B":
            return num_voters_for_B, B_gaining_votes

    return num_voters_for_B

In [26]:
rs_nakamoto_coefficient, final_votes = find_nakamoto_rank_and_slide(rank_and_slide_mechanism=rs_mechanism,
                                 voter_points_dict = rank_and_slide_voters_points)

In [27]:
print(f"The Nakamoto coefficient of Rank-and-Slide is {rs_nakamoto_coefficient}.")

The Nakamoto coefficient of Rank-and-Slide is 84.


In [35]:
84/len(rank_and_slide_voters_points)

0.2420749279538905

In [28]:
final_votes

{'0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'candidate_B': 77.0},
 '0x3e0cf03f718520f30300266dcf4db50ba12d3331': {'candidate_B': 58.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'candidate_B': 56.0},
 '0xcc449df434d886576202eef7c9ef2aa0948c310b': {'candidate_B': 48.0},
 '0x0e950e75b652878e6a3d3ecb6e5797452a918a05': {'candidate_B': 47.0},
 '0x148d424f74c323d3950a8562760aa76e4c58615b': {'candidate_B': 46.0},
 '0xf9278a270d2100ea5e76e8d85ac712b2cb7ec685': {'candidate_B': 45.0},
 '0x47f793e8af738f4b61d7e2d207c6d29c713d1a84': {'candidate_B': 44.0},
 '0x6d465d2081b799770d0ce7e755d8db1665903ffb': {'candidate_B': 39.0},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'candidate_B': 38.0},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'candidate_B': 38.0},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'candidate_B': 38.0},
 '0xc8eba9130f40a0ea9be530407e5b4f7199118864': {'candidate_B': 38.0},
 '0x6e3556f2c8f29559ed67642a1813b177f1871613': {'candidate_B': 38.0},
 '0x278a4bd5a8d09bcf

In Rank and Slide with the current settings, the Nakamoto Coefficient is 84. 

## Nakamoto in GroupHug

In [29]:
gh_mechanism = GroupHug(nft_weights=nft_weights_dict)

In [30]:
def find_nakamoto_in_group_hug(group_hug_mechanism,
                                voter_info_dict):

    # Track how many people have voted for B
    num_voters_for_B = 0
    
    # Create reference dictionary with all votes for A
    all_votes_for_A = {voter: "candidate_A"
                           for voter in voter_info_dict.keys() }
    # Create an identical dictionary 
    B_gaining_votes = {voter: "candidate_A" 
                           for voter in voter_info_dict.keys() }

    for voter in voter_info_dict.keys():
        # Take the default "everyone votes for A" election
        # and change one vote to B
        B_gaining_votes[voter] = "candidate_B"
        
        # Increase the count of how many voters voted for B
        num_voters_for_B = num_voters_for_B + 1
                    
        winner, results = group_hug_mechanism.calculate(voters = voter_info_dict,
                                              voter_choices = B_gaining_votes)
        if winner == "candidate_B":
            return num_voters_for_B, results


    return num_voters_for_B, results

In [31]:
gh_nakamoto_coefficient, final_votes = find_nakamoto_in_group_hug(group_hug_mechanism = gh_mechanism,
                            voter_info_dict = voter_dict)


Experts: {'candidate_A': 94, 'candidate_B': 6}
Intellectuals: {'candidate_A': 99, 'candidate_B': 1}
Participants: {'candidate_A': 100, 'candidate_B': 0}
Community: {'candidate_A': 100, 'candidate_B': 0}

Experts: {'candidate_A': 89, 'candidate_B': 11}
Intellectuals: {'candidate_A': 99, 'candidate_B': 1}
Participants: {'candidate_A': 96, 'candidate_B': 4}
Community: {'candidate_A': 99, 'candidate_B': 1}

Experts: {'candidate_A': 83, 'candidate_B': 17}
Intellectuals: {'candidate_A': 98, 'candidate_B': 2}
Participants: {'candidate_A': 96, 'candidate_B': 4}
Community: {'candidate_A': 99, 'candidate_B': 1}

Experts: {'candidate_A': 78, 'candidate_B': 22}
Intellectuals: {'candidate_A': 98, 'candidate_B': 2}
Participants: {'candidate_A': 96, 'candidate_B': 4}
Community: {'candidate_A': 99, 'candidate_B': 1}

Experts: {'candidate_A': 78, 'candidate_B': 22}
Intellectuals: {'candidate_A': 97, 'candidate_B': 3}
Participants: {'candidate_A': 91, 'candidate_B': 9}
Community: {'candidate_A': 99, 'c

In [32]:
print(f"In GroupHug with the current settings, the Nakamoto Coefficient is {gh_nakamoto_coefficient}. \n This means {100 * gh_nakamoto_coefficient/len(voter_dict)} % of the voters would be enough to win.")

In GroupHug with the current settings, the Nakamoto Coefficient is 93. 
 This means 26.80115273775216 % of the voters would be enough to win.


In [33]:
final_votes

{'candidate_A': 49, 'candidate_B': 51}